In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import statistics
import operator
from xgboost import XGBClassifier
from sklearn.metrics import ndcg_score

# Import Datasets

In [ ]:
train_df = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip')
test_df = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/test_users.csv.zip')

In [ ]:
print(train_df.info())
print(train_df.describe())

In [ ]:
sessions_df = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/sessions.csv.zip')

# Merge sessions data with train and test data
Merging counts of user's action types.

In [ ]:
sessions_df.action_type = sessions_df.action_type.fillna('other')
action_type_size = sessions_df.groupby(['user_id', 'action_type']).size().unstack(fill_value=0).reset_index()
merged_train_df = train_df.merge(action_type_size, how='left', left_on='id', right_on='user_id', suffixes=(False, '_sessions')).drop('user_id', axis='columns').reset_index(drop=True)
merged_test_df = test_df.merge(action_type_size, how='left', left_on='id', right_on='user_id', suffixes=(False, '_sessions')).drop('user_id', axis='columns').reset_index(drop=True)
merged_train_df['has_session_info'] = np.where(merged_train_df['click'].isnull(), 0, 1)
merged_test_df['has_session_info'] = np.where(merged_test_df['click'].isnull(), 0, 1)

# Data preprocessing

In [ ]:
class CustomProcessor(BaseEstimator, TransformerMixin):
        
    def transform(self, X, y=None):
        new_X = X.drop(['date_first_booking'], axis='columns')
        # new_X['date_account_created'] = pd.to_datetime(new_X['date_account_created']).dt.strftime('%Y%m%d').astype('int')
        dac = np.vstack(new_X.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
        new_X['dac_year'] = dac[:,0]
        new_X['dac_month'] = dac[:,1]
        new_X['dac_day'] = dac[:,2]
        new_X = new_X.drop(['date_account_created'], axis=1)
        tfa = np.vstack(new_X.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
        new_X['tfa_year'] = tfa[:,0]
        new_X['tfa_month'] = tfa[:,1]
        new_X['tfa_day'] = tfa[:,2]
        new_X = new_X.drop(['timestamp_first_active'], axis=1)
        av = new_X.age.values
        new_X['age'] = new_X.age.fillna(-1)
        new_X['age'] = np.where(np.logical_or(av<14, av>100), -1, av)
        return new_X

    def fit(self, X, y=None, **fit_params):
        return self

Impute features if they has missing value and one hot encoding for categorical features.

In [ ]:
categorical_cols = [cname for cname in merged_train_df.columns if
                    cname not in ['id','age','timestamp_first_active','date_first_booking','country_destination','date_account_created'] and
                    (merged_train_df[cname].dtype == "object" or cname in ['signup_flow'])]

numerical_cols = [cname for cname in merged_train_df.columns if
                  cname not in ['signup_flow','id','age','date_account_created','timestamp_first_active','date_first_booking','country_destination'] and
                  merged_train_df[cname].dtype != "object"]

print(categorical_cols)
print(numerical_cols)

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ])

In [ ]:
labels = merged_train_df.country_destination
train_X = merged_train_df.drop(['id','country_destination'], axis='columns')
test_id = merged_test_df.id
test_X = merged_test_df.drop(['id'], axis='columns')
le = LabelEncoder()
train_y = le.fit_transform(labels) 

# Grid search
Define cross validation function for NDCG (Normalized discounted cumulative gain) first.

In [ ]:
from sklearn.model_selection import StratifiedKFold
def cross_validation_with_ndcg(pipe, X, y, scorer, cv=5):
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=123)
    # initialize score array
    scores = []
    for train_index, holdout_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[holdout_index]
        y_train, y_test = y[train_index], y[holdout_index]
        pipe.fit(X_train, y_train)
        predict = pipe.predict_proba(X_test)
        y_test = pd.get_dummies(y_test).to_numpy()
        score = scorer(y_test, predict)
        scores.append(round(score, 6))
        print(f'{len(scores)} / {cv} DONE!', end='\r')
    print('')
    print(scores)
    return scores

parameter => n_estimaters, max_depth

In [ ]:
#grid search
n_estimaters_param=[50, 100, 200]
max_depth_param=[3,4,5]
params = [(x, y) for x in n_estimaters_param for y in max_depth_param]
for n_estimaters, max_depth in params:
    xg_model_ = XGBClassifier(max_depth=max_depth,learning_rate=0.1, n_estimators=n_estimaters,verbosity=0,objective='multi:softprob',n_jobs=-1)
    search_pipe = Pipeline([
        ("cp", CustomProcessor()),
        ('preprocessor', preprocessor),
        ("model", xg_model_)
    ])
    print(f'n_estimaters: {n_estimaters}, max_depth: {max_depth}')
    scores = cross_validation_with_ndcg(search_pipe, train_X, train_y, ndcg_score)
    print(statistics.mean(scores))

The best parameter => n_estimaters: 100, max_depth: 5

In [ ]:
xg_model = XGBClassifier(max_depth=5,learning_rate=0.1, n_estimators=100,verbosity=0,objective='multi:softprob',n_jobs=-1)
pipe = Pipeline([
    ("cp", CustomProcessor()),
    ('preprocessor', preprocessor),
    ("model", xg_model)
])
pipe.fit(train_X, train_y)
predict = pipe.predict_proba(test_X)

In [ ]:
ids = []
cts = []
for i in range(len(test_id)):
    idx = test_id[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(predict[i])[::-1])[:5].tolist()

In [ ]:
submission = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
submission.to_csv('./submission.csv',index=False)